In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/notebooks/kagglertw/notebook0921d75084/pairwise_similarity_results.csv')

In [3]:
df

,image1,image2,sim_deep,sim_color,sim_texture,sim_fused
0,107.jpg,825.jpg,1.000000,1.000000,1.000000,1.000000
1,26.jpg,656.jpg,0.993517,0.997827,0.997468,0.995582
2,244.jpg,406.jpg,0.975104,0.986068,0.983508,0.979946
3,486.jpg,741.jpg,0.960892,0.993028,0.997734,0.978137
4,321.jpg,486.jpg,0.952273,0.998766,0.993546,0.974215
...,...,...,...,...,...,...
354056,376.jpg,6.jpg,-0.030412,0.015756,0.486542,0.110368
354057,584.jpg,611.jpg,-0.019151,0.008226,0.462124,0.108012
354058,457.jpg,611.jpg,0.048118,0.006487,0.313458,0.104045
354059,385.jpg,611.jpg,-0.045738,0.051905,0.437985,0.099604


In [ ]:
"""
Create a dictionary: image_id -> list of top 5 most similar images
"""

import pandas as pd
from collections import defaultdict


print(f"Loaded {len(df):,} pairwise comparisons")






image_similarities = defaultdict(list)


for _, row in df.iterrows():
    img1 = row['image1']
    img2 = row['image2']
    sim = row['sim_deep']
    
    
    image_similarities[img1].append((img2, sim))
    
    
    image_similarities[img2].append((img1, sim))





top_5_similar = {}

for image_id, similar_list in image_similarities.items():
    
    top_5 = sorted(similar_list, key=lambda x: x[1], reverse=True)[:5]
    
    
    top_5_similar[image_id] = [img for img, sim in top_5]

print(f"\n✓ Created dictionary for {len(top_5_similar)} images")





print("\n── Examples: Top 5 similar images ──\n")
for i, (image_id, similar_images) in enumerate(list(top_5_similar.items())[:5]):
    print(f"{image_id}:")
    for rank, sim_img in enumerate(similar_images, 1):
        
        score = [s for img, s in image_similarities[image_id] if img == sim_img][0]
        print(f"  {rank}. {sim_img:20s} (similarity: {score:.6f})")
    print()






example_image = "107.jpg"
if example_image in top_5_similar:
    print(f"Top 5 most similar to {example_image}:")
    print(top_5_similar[example_image])
    print()





import json

with open("top_5_similar_images.json", "w") as f:
    json.dump(top_5_similar, f, indent=2)

print("✅ Saved dictionary to 'top_5_similar_images.json'")





df_top5 = pd.DataFrame([
    {
        'image': img,
        'top1': similar[0] if len(similar) > 0 else None,
        'top2': similar[1] if len(similar) > 1 else None,
        'top3': similar[2] if len(similar) > 2 else None,
        'top4': similar[3] if len(similar) > 3 else None,
        
    }
    for img, similar in top_5_similar.items()
])

df_top5.to_csv("top_5_similar_per_image.csv", index=False)
print("✅ Saved DataFrame to 'top_5_similar_per_image.csv'")

print(f"\n── Preview of DataFrame ──")
print(df_top5.head(10))

Loaded 354,061 pairwise comparisons

✓ Created dictionary for 842 images

── Examples: Top 5 similar images ──

107.jpg:
  1. 825.jpg              (similarity: 1.000000)
  2. 197.jpg              (similarity: 0.546009)
  3. 358.jpg              (similarity: 0.492930)
  4. 354.jpg              (similarity: 0.489795)
  5. 38.jpg               (similarity: 0.485642)

825.jpg:
  1. 107.jpg              (similarity: 1.000000)
  2. 197.jpg              (similarity: 0.546009)
  3. 358.jpg              (similarity: 0.492930)
  4. 354.jpg              (similarity: 0.489795)
  5. 38.jpg               (similarity: 0.485642)

26.jpg:
  1. 656.jpg              (similarity: 0.993517)
  2. 2.jpg                (similarity: 0.814034)
  3. 632.jpg              (similarity: 0.787196)
  4. 708.jpg              (similarity: 0.771428)
  5. 311.jpg              (similarity: 0.716156)

656.jpg:
  1. 26.jpg               (similarity: 0.993517)
  2. 2.jpg                (similarity: 0.815901)
  3. 632.jpg     

In [5]:
df_top5

,image,top1,top2,top3,top4
0,107.jpg,825.jpg,197.jpg,358.jpg,354.jpg
1,825.jpg,107.jpg,197.jpg,358.jpg,354.jpg
2,26.jpg,656.jpg,2.jpg,632.jpg,708.jpg
3,656.jpg,26.jpg,2.jpg,632.jpg,708.jpg
4,244.jpg,406.jpg,674.jpg,831.jpg,160.jpg
...,...,...,...,...,...
837,501.jpg,305.jpg,178.jpg,520.jpg,119.jpg
838,404.jpg,435.jpg,722.jpg,806.jpg,355.jpg
839,156.jpg,841.jpg,90.jpg,154.jpg,52.jpg
840,188.jpg,572.jpg,627.jpg,280.jpg,373.jpg


In [6]:
final = pd.read_csv('/kaggle/input/notebooks/kagglertw/final/final.csv')

In [7]:
rp = final[final['right_place']== 0]

In [8]:
rp = rp[rp['prediction_string'] != "-"]

In [9]:
rp_ids = rp['filename'].unique()

In [ ]:
def has_class_1(pred_str):
    if pd.isna(pred_str) or pred_str.strip() in ['-', '']:
        return 0
    
    nums = pred_str.strip().split()
    
    
    if len(nums) % 6 != 0:
        return 0  
    
    classes = [int(float(nums[i])) for i in range(0, len(nums), 6)]
    
    return 1 if 1 in classes else 0


rp['has_class_1'] = rp['prediction_string'].apply(has_class_1)


In [11]:
def has_class_0(pred_str):
    if pd.isna(pred_str) or str(pred_str).strip() in ['-', '']:
        return 0
    
    nums = str(pred_str).strip().split()
    
    if len(nums) % 6 != 0:
        return 0
    
    classes = [int(float(nums[i])) for i in range(0, len(nums), 6)]
    
    return 1 if 0 in classes else 0


In [12]:
rp['has_class_0'] = rp['prediction_string'].apply(has_class_0)

In [13]:
rp = rp[rp['has_class_1'] == 1]

In [14]:
rp = rp[rp['has_class_0'] == 0]

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plot_reference_and_similarities(df, image_id, image_folder):
    """
    Plots reference image and its top5 similar images.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Must contain columns: ['image', 'top1', 'top2', 'top3', 'top4', 'top5']
    image_id : str
        Example: '107.jpg'
    image_folder : str
        Path to folder containing images
    """
    
    
    row = df[df['image'] == image_id]
    
    if row.empty:
        print(f"Image {image_id} not found in dataframe.")
        return
    
    row = row.iloc[0]
    
    
    images = [row['image'], row['top1'], row['top2'], row['top3'], row['top4']]
              
    
    
    fig = plt.figure(figsize=(12, 8))
    
    
    gs = fig.add_gridspec(2, 5)
    
    
    ax_ref = fig.add_subplot(gs[0, 1:4])
    img_path = os.path.join(image_folder, images[0])
    
    if os.path.exists(img_path):
        img = mpimg.imread(img_path)
        ax_ref.imshow(img)
        ax_ref.set_title(f"Reference: {images[0]}", fontsize=14)
    else:
        ax_ref.set_title("Image not found")
    
    ax_ref.axis("off")
    
    
    for i in range(4):
        ax = fig.add_subplot(gs[1, i])
        sim_path = os.path.join(image_folder, images[i+1])
        
        if os.path.exists(sim_path):
            sim_img = mpimg.imread(sim_path)
            ax.imshow(sim_img)
            ax.set_title(f"Top{i+1}", fontsize=10)
        else:
            ax.set_title("Not found")
        
        ax.axis("off")
    
    plt.tight_layout()
    plt.show()


In [16]:
positive_set = set(
    final.loc[final['right_place'] == 1, 'filename']
)

stored_ids = []

for image_id in rp_ids:
    
    row = df_top5[df_top5['image'] == image_id]
    if row.empty:
        continue
    
    similar_images = set(
        row[['top1',]].values.flatten()
    )
    
    if len(similar_images & positive_set) > 0:
        stored_ids.append(image_id)

print("Stored IDs:", stored_ids)

Stored IDs: ['194.jpg', '238.jpg', '260.jpg', '3.jpg', '355.jpg', '435.jpg', '481.jpg', '517.jpg', '573.jpg', '633.jpg', '634.jpg', '668.jpg', '726.jpg', '743.jpg', '748.jpg', '784.jpg', '795.jpg', '827.jpg']


In [17]:
def has_class_0(pred_str):
    if pd.isna(pred_str) or str(pred_str).strip() in ['-', '']:
        return 0
    
    nums = str(pred_str).strip().split()
    
    if len(nums) % 6 != 0:
        return 0
    
    classes = [int(float(nums[i])) for i in range(0, len(nums), 6)]
    
    return 1 if 0 in classes else 0


In [18]:
final['has_class_0'] = final['prediction_string'].apply(has_class_0)

In [ ]:

positive_set = set(
    final.loc[final['has_class_0'] == 1, 'filename']
)

has_face_sim = []
no_face_sim = []

for image_id in stored_ids:
    
    row = df_top5[df_top5['image'] == image_id]
    if row.empty:
        continue
    
    
    similar_images = set(
        row[['top1','top2','top3','top4']].values.flatten()
    )
    
    
    if similar_images & positive_set:
        has_face_sim.append(image_id)
    else:
        no_face_sim.append(image_id)

print("Have similar with class 1:", has_face_sim)
print("No similar with class 1:", no_face_sim)

Have similar with class 1: ['194.jpg', '238.jpg', '260.jpg', '3.jpg', '355.jpg', '435.jpg', '481.jpg', '517.jpg', '573.jpg', '633.jpg', '634.jpg', '668.jpg', '743.jpg', '748.jpg', '784.jpg', '795.jpg', '827.jpg']
No similar with class 1: ['726.jpg']


In [ ]:
final.loc[final['filename'].isin(has_face_sim), 'right_place'] = 1

for image_id in no_face_sim:
    
    
    row = df_top5[df_top5['image'] == image_id]
    if row.empty:
        continue
    
    similar_images = row[['top1','top2','top3','top4']].values.flatten()
    
    
    final.loc[final['filename'].isin(similar_images), 'right_place'] = 0


In [21]:
old = pd.read_csv('/kaggle/input/notebooks/kagglertw/final/final.csv')

In [22]:
final = final[old.columns]

In [ ]:
merged = final.merge(old, on='filename', how='outer', suffixes=('_1', '_2'))


diff_rows = merged[
    (merged['right_place_1'] != merged['right_place_2']) |
    (merged['prediction_string_1'] != merged['prediction_string_2'])
]

In [24]:
final.to_csv('final.csv' , index = False)